# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_69125ad0f40c8191a974c7c5305d28ca


### Upload Files

In [7]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-K3yMq3q4g8qSSrXqjLeEo4


### Attach File to Vector Store

In [8]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-K3yMq3q4g8qSSrXqjLeEo4


### Query the Vector Store

In [9]:
query = "the latest development in generativeAI"

In [10]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6029813181076801
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5984074448147717
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.5911156596667607
⚡\n\nThe global #GenerativeAI market will hit $1.18 B this year — act now or get left behind. 🚀\n\n🔗
 Relevant score: 0.5728063202962544
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.5622353955032359


## OpenAI Response API

### Simple Response

In [11]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [12]:
display(Markdown(simple_response.output_text))

As of the latest updates in 2023, there have been several exciting developments in the field of generative AI:

1. **Multimodal Models**: Models like GPT-4 have advanced to handle text, image, and other data types, allowing seamless integration across different modalities.

2. **Improved Fine-Tuning**: Techniques for fine-tuning models on specific tasks have become more sophisticated, enabling better customization and efficiency.

3. **AI-Assisted Creativity**: Generative AI is increasingly being used in creative industries, aiding in music, art, and content creation with tools that offer more intuitive user interfaces.

4. **Ethical AI and Safety**: There is growing emphasis on making generative AI more ethical, with improved frameworks for bias detection and reduction, as well as mechanisms for ensuring responsible deployment.

5. **AI in Gaming**: Generative AI is being used to create more dynamic and immersive gaming experiences, including procedural content generation and adaptive narratives.

6. **Open-Source Collaboration**: There is a growing trend toward open-source contributions, expanding accessibility and encouraging innovation through community-driven projects.

These advancements highlight the rapid evolution and broadening applications of generative AI across various sectors.

### File Search Response

In [13]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [14]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis tool using generative AI, SAP BTP, and AWS Bedrock to assess risk and boost resilience.

3. **Market Growth**: The global generative AI market is projected to reach $1.18 billion this year, highlighting the rapid growth and adoption of these technologies.

4. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of cinematic videos, music, and other content with minimal human intervention.

5. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing teams to build custom large language models for enhanced customer engagement and streamlined processes.

These developments illustrate the expanding role of generative AI across various sectors, from creative industries to enterprise solutions.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [15]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [16]:
display(Markdown(web_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025:

OpenAI Emerges with GPT‑5  
OpenAI launched **GPT‑5** on August 7, 2025, a cutting-edge multimodal foundation model that significantly surpasses prior versions in benchmark performance. This model is accessible through ChatGPT, Microsoft Copilot, and via the OpenAI API ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai)).

OpenAI also released two notable open-weight models earlier in August 2025:
- **gpt‑oss‑20b**, optimized for local use on consumer hardware like 16 GB RAM machines and Snapdragon processors.
- **gpt‑oss‑120b**, a more powerful variant designed for running on RTX PRO GPUs. These models democratize AI by enabling local, offline usage and supporting chain-of-thought reasoning ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/openai-chatgpt/openai-launches-two-gpt-models-theyre-not-gpt-5-but-they-run-locally-on-snapdragon-pcs-and-nvidia-rtx-gpus?utm_source=openai)).

Anthropic’s Elevated Reasoning Models  
In February 2025, Anthropic introduced **Claude 3.7 Sonnet** featuring an “extended thinking mode.” This enables the model to reflect on its responses before outputting them, improving performance across domains such as math, physics, and coding ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/anthropic-launches-advanced-ai-hybrid-reasoning-model-2025-02-24/?utm_source=openai)).

By May 2025, Anthropic raised the bar further with **Claude Opus 4** and **Claude Sonnet 4**:
- **Claude Opus 4** excels in deep reasoning and handling complex, long-running coding tasks.
- **Claude Sonnet 4** enhances instruction-following and tool use, and is now available even to free-tier users ([linkedin.com](https://www.linkedin.com/pulse/top-5-generative-ai-news-updates-from-week-21-2025-18th-shankar-vpgpc?utm_source=openai)).

Google DeepMind Boosts Multimodal Capabilities  
Google DeepMind released **Gemini 2.5 Pro** and **Flash** models in early 2025, introducing improved reasoning, coding, and multimodal proficiency. Gemini 2.5 Pro includes “Deep Think” mode for handling complex tasks, along with native multimodal features like audio output ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)).

In August 2025, Google’s **Nano Banana** (officially Gemini 2.5 Flash Image) emerged as a viral sensation. This highly advanced image-editing and generation model produces photorealistic “3D figurine” style images, enabling seamless modifications via natural language, multi-image fusion, and watermarking for AI-generated content identification. It quickly gained popularity, accumulating millions of users and image edits ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai)).

Tencent Advances Text-to-3D Generation  
On March 18, 2025, Tencent released five open-source **Hunyuan3D‑2.0** models capable of converting text and images into 3D visuals in as little as 30 seconds, focusing on designers and game developers. This underscores China's expanding influence and competitiveness in the generative AI space ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/tencent-expands-ai-push-with-open-source-3d-generation-tools-2025-03-18/?utm_source=openai)).

China's Kuaishou also pushed forward with **Kling AI**, a text-to-video model. The most recent version, **Kling 2.1**, launched in May 2025, offers improved video generation with multiple quality modes ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kling_AI?utm_source=openai)).

Moves Toward On‑Device and Democratized AI  
Beyond OpenAI’s open-weight models, Google rolled out the **AI Edge Gallery**, an experimental Android app that allows users to run AI models locally, even offline. The app lets users download and run open-source models (e.g., Gemma 3n) from Hugging Face, aiming for faster, more private AI interactions ([linkedin.com](https://www.linkedin.com/pulse/top-5-generative-ai-news-updates-from-week-22-2025-25th-shankar-uahbc?utm_source=openai)).

Robotics Meets Generative AI  
In May 2025, Hugging Face unveiled two open-source humanoid robots—the full-size **HopeJR** and compact **Reachy Mini**. These democratize AI-driven robotics by offering affordable, programmable platforms for creators and researchers alike ([linkedin.com](https://www.linkedin.com/pulse/top-5-generative-ai-news-updates-from-week-22-2025-25th-shankar-uahbc?utm_source=openai)).

Academic Directions & Tools  
On the academic and tooling front:
- **GAICo**, debuted in August 2025, is an open-source Python framework designed to standardize evaluation of generative AI outputs across text, audio, and image domains ([arxiv.org](https://arxiv.org/abs/2508.16753?utm_source=openai)).
- **Chronologically Consistent Generative AI**, introduced in October 2025, offers LLMs trained only on data preceding a clear cutoff date. This reduces lookahead bias and ensures replicability and forecast reliability in prediction tasks ([arxiv.org](https://arxiv.org/abs/2510.11677?utm_source=openai)).

Summary of the Major Generative AI Milestones in 2025

OpenAI:
- GPT‑5 (August 2025)
- Open-weight gpt‑oss-20b and gpt‑oss-120b (August 2025)

Anthropic:
- Claude 3.7 Sonnet (February 2025)
- Claude Opus 4 and Sonnet 4 (May 2025)

Google DeepMind:
- Gemini 2.5 Flash and Pro (early 2025)
- Nano Banana / Gemini 2.5 Flash Image (August 2025)

Tencent / Kuaishou:
- Hunyuan3D‑2.0 3D models (March 2025)
- Kling AI 2.1 text-to-video (May 2025)

On-device AI:
- AI Edge Gallery app (May 2025)

Robotics:
- HopeJR and Reachy Mini by Hugging Face (May 2025)

Academic frameworks:
- GAICo evaluation library (August 2025)
- Chronologically consistent LLMs (October 2025)

---

Let me know if you'd like deep dives into any of these developments—GPT‑5’s capabilities, Nano Banana’s cultural impact, hybrid reasoning in Anthropic models, or tools like GAICo for assessing AI outputs and more!

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [17]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of

### Continue Query with Web Search

In [18]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [19]:
display(Markdown(continue_search_response.output_text))

Here are some *recent developments* in generative AI — spanning industry launches, major research trends, regulation, and emerging applications — as of early November 2025:

###  OpenAI’s Agentic Security Researcher: **Aardvark** (Oct 30, 2025)  
- OpenAI introduced **Aardvark**, a GPT-5–powered AI agent designed to autonomously identify security vulnerabilities in codebases. In private beta, it already achieves approximately **92% recall** on seeded "golden" repositories and has contributed to **10 CVE-tracked disclosures**. The system integrates naturally with developer workflows, offering explanations and remediation advice.  
  ([aiwebbiz.com](https://aiwebbiz.com/blog/top-5-ai-news-of-the-week-november-2025/?utm_source=openai))

###  Google’s Expansive AI Integrations (Nov 1–7, 2025 updates)  
- **Sora for Android**: OpenAI’s AI video app hit ~470,000 downloads on day one (compared to ~360,000 on iOS), reflecting a 327% stronger debut.  
- **DS STAR**: A new multi-agent framework for processing unstructured or mixed-format data (CSV, JSON, Markdown, free text) into executable Python code — enabling near-zero human intervention in data workflows.  
- **Meta’s Vibes**: Expanded its AI-generated short-video platform to Europe, delivering automatically generated content in a TikTok-style feed.  
- **Gemini on Google Maps**: Rolling out voice-powered generative assistance for destination search globally, across Gemini-compatible Android and iOS devices.  
- **HeyGen AI Video Translator**: Introduced hyper-realistic localization, complete with matching lip movements and tone—available via web, mobile, and API platforms.  
- **Gemini 3 Pro Preview** spotted on Vertex AI, expected to support a 1 million token context window.  
- **Deep Research for Gemini**: New feature to extract and compile information from Gmail, Drive, and Chat into research-style summaries.  
- **Chrome AI Mode**: A quick-access AI button now appears on new tabs (initially in the U.S., expanding to 160 countries).  
- **Nano Banana2 (GEMPIX2)**: Google's next-gen image model is hinted to launch soon, promising even more advanced image generation within Gemini.  
- **ClickUp 4.0**: Launched with integrated AI agents and a redesigned interface, merging task management, docs, messaging, and scheduling in an AI-supported environment.  
  ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

###  Adobe's AI Innovation: **Firefly Boards** (Sept 26, 2025)  
- Adobe globally rolled out **Firefly Boards**, an AI-first collaborative platform for brainstorming and multimedia creation. The tool integrates models not just from Adobe but also partners like Google, Runway, Luma AI, Black Forest Labs, and Moonvalley.  
  ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/artificial-intelligence/adobe-launches-firefly-boards-globally-with-new-ai-video-models-and-features/articleshow/124153220.cms?utm_source=openai))

###  Gaming Industry Embraces Generative AI  
- According to a study, **one in five games released on Steam in 2025** use generative AI—**7,818 games** in total, representing about 7% of the entire library. This shows a **681% year-over-year increase** from 2024, with AI mainly used for visuals (60%), audio, narrative text, marketing, coding, and even live gameplay dynamics.  
  ([tomshardware.com](https://www.tomshardware.com/video-games/pc-gaming/1-in-5-steam-games-released-in-2025-use-generative-ai-up-nearly-700-percent-year-on-year-7-818-titles-disclose-genai-asset-usage-7-percent-of-entire-steam-library?utm_source=openai))

###  Google DeepMind’s **World Modeling AI Team** (Jan 7, 2025)  
- DeepMind formed a dedicated team, led by Tim Brooks, to build **“world models”** for realistic physical simulation environments. These models will support video games, robotics training, and possibly AGI. The team will integrate with projects like Gemini, Veo, and the Genie world model.  
  ([theverge.com](https://www.theverge.com/2025/1/7/24338053/google-deepmind-world-modeling-ai-team-gaming-robot-training?utm_source=openai))

###  Chinese AI Landscape: Tencent & Baidu  
- **Tencent** released five **Hunyuan3D‑2.0** open-source models in March 2025, capable of transforming text and images into 3D visuals in 30 seconds, emphasizing high precision and low-cost accessibility.  
  ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/tencent-expands-ai-push-with-open-source-3d-generation-tools-2025-03-18/?utm_source=openai))  
- **Baidu**, at its developer conference, criticized the waning demand for text-only models like DeepSeek's R1, touting its new multimodal models — **Ernie 4.5 Turbo** and **X1 Turbo** — which support text, audio, image, and video. They also unveiled a new AI agent app (**Xinxiang**) and a 30,000-chip computing cluster.  
  ([ft.com](https://www.ft.com/content/c462fbd1-1672-4d8f-bd91-c3aa185d2418?utm_source=openai))

###  Choosing What Matters  
Collectively, these developments show generative AI's transition from novelty to integral infrastructure:

- Autonomous AI agents like Aardvark are reshaping cybersecurity.
- Seamless multi-modal capabilities are embedding AI into tools—from mapping to code generation, video localization, and workplace productivity.
- The game, creative, and enterprise sectors are rapidly adapting, integrating GenAI into narrative, design, and collaboration.
- Foundational innovations—like world modeling, 3D generation, and massive AI chips—are enabling complex simulations and large-scale deployments.

---

If you’d like me to explore any of these in depth—say, technology comparisons, company strategies, governance implications, or technical breakdowns—feel free to ask!

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [20]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [21]:
display(Markdown(combined_search_response.output_text))

Here’s a comprehensive and up-to-date overview of the **latest developments in generative AI** as of November 10, 2025. This analysis draws on recent news, model releases, market trends, and research breakthroughs.

---

##  Major Model Releases and Innovations

- **OpenAI GPT‑5**  
  Released on **August 7, 2025**, GPT‑5 is a multimodal foundation model that unifies reasoning and non-reasoning capabilities under a single interface. It is accessible via ChatGPT, Microsoft Copilot, and the OpenAI API, and represents a significant leap in performance across benchmarks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **OpenAI o4‑mini**  
  Launched on **April 16, 2025**, this lightweight, multimodal model supports both text and image inputs. It includes a high-accuracy variant (o4‑mini‑high) for paid users and is designed for tasks like whiteboard sketch analysis and infrastructure data interpretation. ([en.wikipedia.org](https://en.wikipedia.org/wiki/OpenAI_o4-mini?utm_source=openai))

- **OpenAI GPT‑4.1**  
  Released on **April 14, 2025**, GPT‑4.1 (along with mini and nano variants) improved coding and reasoning capabilities and was made available to ChatGPT Plus and Pro subscribers. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-4.1?utm_source=openai))

- **Google DeepMind’s Gemini 2.5 and “Nano Banana”**  
  - Gemini 2.5 Pro and Flash models, featuring enhanced reasoning, coding, and “Deep Think” capabilities, became generally available on **June 17, 2025**. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai))  
  - The image-focused variant, **Nano Banana** (Gemini 2.5 Flash Image), launched on **August 26, 2025**, and quickly went viral for its photorealistic 3D figurine-style outputs. It supports multi-image fusion, subject consistency, and SynthID watermarking. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Google Gemini Diffusion**  
  An experimental model that applies diffusion techniques to text generation, enabling simultaneous generation of large text blocks with mid-process error correction. It achieves speeds up to **1,479 tokens per second**, far exceeding Gemini 2.5 Flash (~400 tps) and GPT‑4o (~150 tps). ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai))

- **Alibaba Qwen3**  
  Released in **April 2025**, this open-source model family (Apache 2.0 license) includes dense and Mixture-of-Experts variants, offering strong performance in coding, math, and instruction-following tasks. ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/generative-ai-digest-a-wave-of-notable-ai-model-launches?utm_source=openai))

- **Anthropic Claude Haiku 4.5**  
  A compact model delivering flagship-level reasoning and coding performance at significantly lower cost and higher speed. It features a 200K token context window and agentic capabilities like “using computers” to automate tasks. ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai))

- **xAI’s Grok 3 and Grok 3 Mini**  
  Launched in early 2025, Grok 3 offers enhanced reasoning and analytics, while Grok 3 Mini provides a faster, lighter alternative optimized for everyday chatbot interactions. ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai))

- **Anthropic Claude 3.7 Sonnet**  
  Introduced with an “Extended Thinking Mode” that allows the model to self-reflect and refine responses before output, improving factual accuracy and structure. ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai))

- **OpenAI Deep Research & Perplexity Deep Research**  
  These features enable AI to autonomously browse the web, verify facts, and generate detailed, cited reports—integrated into ChatGPT and Perplexity’s premium offerings. ([analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2025/03/generative-ai-launches-of-february/?utm_source=openai))

- **Baidu Ernie 4.5 and Ernie X1**  
  Recently released multimodal and reasoning models that outperform competitors on benchmarks like CCBench and OCRBench. Ernie 4.5 is slated to become open-source starting **June 30, 2025**. ([globenewswire.com](https://www.globenewswire.com/news-release/2025/04/17/3063915/0/en/Applied-Generative-AI-Course-Launched-by-Interview-Kickstart-2025-Best-GenAI-Course-With-Agentic-AI-Projects-For-Top-AI-Jobs-at-Google-Meta-Netflix-Microsoft-OpenAI-Nvidia.html?utm_source=openai))

---

##  Integration into Products & Ecosystems

- **TIME AI Agent**  
  Launched **today**, this AI-powered platform enhances reader interaction with TIME’s journalism. It offers summaries, audio reports, translations, and interactive exploration, built in collaboration with Scale AI and designed with strong editorial standards. ([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

- **Microsoft MAI‑Image‑1**  
  Released **5 days ago**, this proprietary text-to-image generator is integrated into Bing Image Creator and Copilot Audio Expressions. It offers fast, photorealistic image generation and complements OpenAI models within Microsoft’s ecosystem. ([windowscentral.com](https://www.windowscentral.com/artificial-intelligence/microsoft-copilot/microsoft-launches-mai-image-1?utm_source=openai))

- **Google Gemini in Maps & Deep Research**  
  Gemini AI is being integrated into Google Maps as a voice assistant for intuitive navigation. Additionally, a “Deep Research” feature now enables Gemini to generate reports using data from Gmail, Drive, and Chat. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **Meta Vibes Expansion**  
  Meta’s AI-generated short video platform, Vibes, has expanded into Europe, offering automated, TikTok-style content creation. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **HeyGen AI Video Translator**  
  This tool delivers hyper-realistic localization by matching tone, expressions, and lip movements in translated videos. Available via web, iOS, and API. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **ClickUp 4.0**  
  Released recently, this update includes AI agents and a redesigned UI, integrating task management, collaboration, messaging, scheduling, and enterprise search into a unified platform. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

- **OpenAI Sora App**  
  The Android version of Sora, OpenAI’s AI video app, launched on **November 6, 2025**, achieving nearly **470,000 downloads** on its first day—surpassing the iOS debut. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

##  Market Trends & Responsible AI

- **Generative AI Market Growth**  
  The generative AI market experienced triple-digit growth across hardware, foundation models, and development platforms in 2024. Spending is expected to exceed **US$400 billion in 2025**. ([businesswire.com](https://www.businesswire.com/news/home/20250825682581/en/Generative-AI-Market-Report-2025-GenAI-Market-Experienced-Triple-digit-growth-Rates-in-All-Three-Major-Segments-Spanning-GenAI-Hardware-Foundation-Models-and-Development-Platforms---ResearchAndMarkets.com?utm_source=openai))

- **Enterprise Adoption & Agentic AI**  
  According to the 2025 McKinsey Global Survey, **23%** of organizations are scaling agentic AI systems, and **39%** are experimenting with them. High performers are more likely to redesign workflows, invest over 20% of digital budgets in AI, and define human validation processes. ([mckinsey.com](https://www.mckinsey.com/capabilities/quantumblack/our-insights/the-state-of-ai?utm_source=openai))

- **Responsible Generative AI Playbook**  
  UC Berkeley’s Responsible AI Initiative released a playbook outlining 10 best practices (five for business leaders, five for product managers) to guide responsible GenAI development and use. ([weforum.org](https://www.weforum.org/stories/2025/06/responsible-generative-ai-product-development-use/?utm_source=openai))

---

##  Research & Emerging Concepts

- **Chronologically Consistent Generative AI**  
  A new family of instruction-following models trained only on data before a defined cutoff, eliminating lookahead bias and ensuring replicability. ([arxiv.org](https://arxiv.org/abs/2510.11677?utm_source=openai))

- **Agentic AI Networking (AgentNet)**  
  A proposed framework for multi-agent collaboration in 6G networks, enabling autonomous, generative foundation models to interact, learn, and transfer knowledge in industrial and metaverse environments. ([arxiv.org](https://arxiv.org/abs/2503.15764?utm_source=openai))

- **Interactive Generative Video for Game Engines**  
  A position paper proposes using generative video models as the basis for next-gen game engines, enabling physics-aware, interactive, and memory-capable content generation. ([arxiv.org](https://arxiv.org/abs/2503.17359?utm_source=openai))

---

##  Summary

The **latest developments in generative AI** as of November 10, 2025, reflect a dynamic landscape marked by:

- **Advanced multimodal models** (e.g., GPT‑5, Gemini 2.5, Claude Haiku 4.5)
- **Innovative product integrations** (e.g., TIME AI Agent, MAI‑Image‑1, Sora)
- **Rapid market expansion** and enterprise adoption
- **Growing emphasis on responsible AI practices**
- **Cutting-edge research** in bias mitigation, agentic systems, and generative video

If you'd like to explore any of these developments in more detail—such as technical capabilities, enterprise use cases, or ethical considerations—feel free to ask!

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [23]:
vector_store = client.vector_stores.create(
    name="my_vector_store2"
)
vector_store_id = vector_store.id
print(vector_store_id)
with open('mysample_text.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

vs_69126a8428bc81919c767883da385402
file-3p6JUPzA8ibKFYVPsFhSNj


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [24]:
query = "Using my uploaded notes and the latest web information, summarize the current trends on this topic."

combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input=query,
    temperature=0,
    instructions=(
        "Retrieve the results from the file search first, "
        "and use the web search tool to expand the results with news resources."
    ),
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
        },
        {
            "type": "web_search"
        }
    ]
)

# Display the merged output as nicely formatted markdown
display(Markdown(combined_search_response.output_text))

Here’s a refined summary of the **current trends** based on your uploaded notes and the latest web-sourced insights. I’ve structured the analysis into thematic sections, each supported by at least two citations per paragraph, and organized for clarity and depth.

---

## 1. Generative AI & Autonomous Agents  
- Generative AI continues to dominate technological discourse in 2025, evolving from content generation to full-scale automation of workflows. Businesses are increasingly integrating AI into customer service, design, and strategic planning, with tools like GPTs and multimodal systems enabling hyper-personalized experiences across industries ([appinventors.com](https://appinventors.com/blog/new-technology-trends/?utm_source=openai)).  
- Autonomous AI agents—capable of handling end-to-end tasks—are emerging as a major trend. The autonomous AI market is projected to reach USD 11.79 billion by 2026, growing at a CAGR above 40% through 2035 ([simplilearn.com](https://www.simplilearn.com/top-technology-trends-and-jobs-article?utm_source=openai)).

## 2. AI in Cybersecurity & Deepfake Defense  
- The rise of generative AI has heightened cybersecurity risks, prompting organizations to adopt AI-driven defenses. Nearly all surveyed organizations have encountered Gen AI-related security issues, with 44% of executives citing it as a top cybersecurity concern ([capgemini.com](https://www.capgemini.com/news/press-releases/technovision-top-5-tech-trends-to-watch-in-2025/?utm_source=openai)).  
- Deepfake threats are accelerating the need for AI-powered detection and content authentication, with blockchain and other technologies being deployed to ensure data integrity ([infotech.com](https://www.infotech.com/research/2025-tech-trends-report-info-tech-research-group-unveils-new-insights-on-the-future-of-ai-quantum-computing-and-cybersecurity?utm_source=openai)).

## 3. Quantum Computing & Post-Quantum Security  
- Quantum computing is transitioning from theory to practical application. Many organizations are experimenting with quantum proofs of concept, and 55% of executives expect it to be a top technology in 2025 ([capgemini.com](https://www.capgemini.com/news/press-releases/technovision-top-5-tech-trends-to-watch-in-2025/?utm_source=openai)).  
- In parallel, post-quantum cryptography is gaining urgency. About 31% of advanced IT departments plan to invest in quantum-safe encryption by the end of 2025 to guard against future threats ([infotech.com](https://www.infotech.com/research/2025-tech-trends-report-info-tech-research-group-unveils-new-insights-on-the-future-of-ai-quantum-computing-and-cybersecurity?utm_source=openai)).

## 4. Edge Computing, 5G-Advanced & IoT  
- Edge computing is becoming essential for real-time analytics and low-latency applications. Gartner predicts that by 2025, 75% of enterprise-generated data will be processed outside centralized data centers ([devsu.com](https://devsu.com/blog/6-game-changing-tech-trends-shaping-2025-ai-cloud-?utm_source=openai)).  
- The rollout of 5G-Advanced (5.5G) is underway, offering speeds up to 10 Gbit/s, ultra-low latency, and support for massive IoT deployments. Finland and China have already begun commercial deployments in early 2025 ([en.wikipedia.org](https://en.wikipedia.org/wiki/5G?utm_source=openai)).

## 5. Blockchain, Web3 & Decentralized Systems  
- Blockchain is expanding beyond cryptocurrency into supply chain transparency, digital identity, and secure data sharing. Deloitte estimates blockchain could generate up to $3 trillion in business value by 2025 ([devsu.com](https://devsu.com/blog/6-game-changing-tech-trends-shaping-2025-ai-cloud-?utm_source=openai)).  
- Web3 and decentralized applications are gaining traction, enabling peer-to-peer interactions, creator royalties via NFTs, and transparent supply chains ([medium.com](https://medium.com/%40panelsai/top-10-technology-trends-to-watch-in-2025-2026-8ef36e36e89d?utm_source=openai)).

## 6. Wearables & Earable Technology  
- Wearable technology is evolving toward seamless integration with AI, IoT, and AR, emphasizing personalization, real-time adaptability, and sustainability through modular and eco-friendly designs ([arxiv.org](https://arxiv.org/abs/2502.05797?utm_source=openai)).  
- Earable devices—wearables positioned in or around the ear—are rapidly advancing from audio accessories to multifunctional systems for health monitoring and contextual interaction. Research since 2022 highlights novel sensing capabilities and expanding applications ([arxiv.org](https://arxiv.org/abs/2506.05720?utm_source=openai)).

## 7. Sustainability, Circular Economies & Clean Tech  
- Sustainable and circular economy models are gaining momentum, with businesses and governments adopting eco-friendly practices, green supply chains, and zero-waste goals ([theacademicnetwork.net](https://theacademicnetwork.net/top-10-emerging-trends-and-research-insights-shaping-2025/?utm_source=openai)).  
- Clean energy, particularly nuclear power, is resurging to meet the high energy demands of AI and other technologies. Small modular reactors (SMRs) are becoming a focal point in the clean tech agenda ([sogeti.us](https://www.sogeti.us/research-and-insight/top-tech-trends-of-2025-ai-powered-everything/?utm_source=openai)).

## 8. Business & Market Trends  
- In the psychotherapy services market, emerging trends are shaping industry dynamics and growth projections through 2032 ([openpr.com](https://www.openpr.com/news/4261816/psychotherapy-service-market-2025-2032-emerging-trends?utm_source=openai)).  
- The in-mold labels market is expected to grow significantly, from USD 2.57 billion in 2026 to much higher by 2035 ([globenewswire.com](https://www.globenewswire.com/news-release/2025/11/10/3184819/0/en/In-Mold-Labels-Market-Size-Trends-Segments-Share-and-Companies-2025-35.html?utm_source=openai)).  
- In online gambling, users are demanding more control, enhanced security, and greater variety in iGaming experiences ([thisdaylive.com](https://www.thisdaylive.com/2025/11/10/new-trends-in-online-gambling-for-2025-expert-insights-from-svitua/?utm_source=openai)).

---

### Summary  
The convergence of **generative AI**, **autonomous agents**, and **quantum computing** is reshaping enterprise workflows and security paradigms. **Edge computing** and **5G-Advanced** are enabling real-time, low-latency applications, while **blockchain** and **Web3** are redefining trust and decentralization. Wearables and earables are becoming more intelligent and integrated, and sustainability is driving innovation in clean tech and circular economies. Across industries—from mental health to packaging and gaming—market trends reflect evolving consumer expectations and technological capabilities.

Let me know if you'd like a deeper dive into any specific area or how these trends intersect with your notes.